# The Reduced Row Echelon Algorithm (RREF)

The RREF algorithm is the swiss army knife of Linear Algebra. It is a process that is used continuously to solve problems from each section of the 313 course. Naturally the steps lend themselves nicely to computer scripts which perform RREF on some specified matrix.

   ## steps for RREF
    Begin with the leftmost nonzero column. This is a pivot column. The pivot
    position is at the top.
    
    Select a nonzero entry in the pivot column as a pivot. If necessary, interchange
    rows to move this entry into the pivot position.
    
    Use row replacement operations to create zeros in all positions below the pivot.
    
    Cover (or ignore) the row containing the pivot position and cover all rows, if any,
    above it. Apply steps 1–3 to the submatrix that remains. Repeat the process until
    there are no more nonzero rows to modify.
    
    Beginning with the rightmost pivot and working upward and to the left, create
    zeros above each pivot. If a pivot is not 1, make it 1 by a scaling operation.
    

The name can be explained best from right to left. Echelon form is an arrangement of entries in a matrix such that each row's entry is to the right (1 or more columns to the right) of the entry in the row above. Reduced Row describes the process in which we apply row operations (see elementary matrix notebook for row operations explained) until reach a point where the matrix is in echelon form, and then reduce (divide) each row such that the first entry in the row is a 1.

To get started simply run the cell below (ctrl-Enter or shift-Enter) and we will import the necessary pieces to be used in the RREF script I wrote. 

In [1]:
from __future__ import division
import random
from sympy import *
x, y, z, t = symbols('x y z t')
k, m, n = symbols('k m n', integer=True)
f, g, h = symbols('f g h', cls=Function)
init_printing()

## Helper functions

To make the process of calculating the RREF version of a matrix easier I have written several of the important sub processes into functions of their own.

#### Target swap
This is a function that allows for the swapping of specific rows in our matrix

#### Find nonzero ind
This function finds the first index value in a column below the current row, col position (more on this later).

#### Add mult row
This is a short function that gives back the correct numerical value that a row needs to be multiplied by in order to zero out a position in some other row


In [ ]:
def target_swap(mtx,row,lst):
    ret_mtx = mtx.copy()
    rep_ind = find_nonzero_ind(lst,row)
    if rep_ind:
        rep_row = ret_mtx[rep_ind,:]
        ret_mtx[rep_ind,:] = ret_mtx[row,:]
        ret_mtx[row,:] =rep_row
        return ret_mtx
    return False

def find_nonzero_ind(lst,row,backwards=False):
    if backwards:
        lst_to_bool = [x == 0 if i < row else True for i,x in enumerate(lst)]
    else:
        lst_to_bool = [x == 0 if i > row else True for i,x in enumerate(lst)]
    if False not in lst_to_bool:
        return False
    return lst_to_bool.index(False)

def add_mult_row(mtx,piv_row,other):
    return solve(piv_row*x+ other,x)[0]

### The main event Short EF

The name here is taken from the fact that a previous version of the RREF was attempted that became a sprawling monster of code, so by comparison this is the shorter of the functions to calculate Echelon Form.

This function runs by looping over the potential pivot positions in a matrix. A pivot position is just another way of describing the eventual nonzero entries in a RREF'd matrix. We will one by one check possible row and column positions for these pivots.

What follows next is a section that determines the current state of the matrix by looking at our current column in list form. So, if we have as many zeros in a column as we have rows in the matrix then we need to shift to the next column to look for pivots. Also, if there is an entire row of zeros we need to shift one column over. If the current pivot search position is a zero, then we will perform the row swapping function from above. Lastly, if there is the correct number of zeros below the position in the matrix we are investigating, and that entry isn't zero the we have found a pivot and need to continue with the next row next column.


        if col_list.count(0) == cop.rows or col_list[row:].count(0) > cop.rows -1 -row:
            col +=1
            continue
        if cop[row,col] == 0 :
                cop = target_swap(cop,row,col_list)
                try_cap -=1
                continue
        if col_list[row:].count(0) == cop.rows -1 -row:
            pivots.append((row,col))
            col +=1
            row +=1
            continue
            
Now, if none of these conditionals applies then we are in the position in which we have found a potential non-zero pivot position, and there just happens to be another non-zero entry below in the column. First step is to locate that entry below using the helper function "find nonzero ind", and then we determine the scalar multiple that the current pivot row needs to be multiplied by so that when these two rows are added, we zero out the entry below.

            rep_ind = find_nonzero_ind(col_list,row) #makes sure to not include the potential pivot row in the search but keeps the indexing constant with mtx
            multiple_factor = add_mult_row(cop,cop[row,col],cop[rep_ind,col])
            cop[rep_ind,:] += cop[row,:]*multiple_factor
            
With this all taken care of the loop restarts until all the potential pivot positions have been evaluated.
 
The second to last step here is to reduce each of the rows by dividing the whole thing by whatever their leading (first non-zero from the left edge) entry is.
        mtx[row,:] = mtx[row,:]/mtx[row,col]
The final aesthetic step is to re-arrange the rows into the echelon form.
        rep_ind = find_nonzero_ind(col_list,row,True) 
        multiple_factor = add_mult_row(mtx,mtx[row,col],mtx[rep_ind,col])
        mtx[rep_ind,:] += mtx[row,:]*multiple_factor
        col_list = list(mtx[:,col])

In [ ]:
def shortEF(mtx):
    pivots = []
    cop = mtx.copy()
    row,col = 0,0
    try_cap = cop.rows + 30
    while row < cop.rows and col < cop.cols: #took off -1 from cop.rows mighthave broken stuff
        col_list = list(cop[:,col])
        if try_cap < 0:
            return (row,col)
        if col_list.count(0) == cop.rows or col_list[row:].count(0) > cop.rows -1 -row:
            col +=1
            continue
        if cop[row,col] == 0 :
                cop = target_swap(cop,row,col_list)
                try_cap -=1
                continue
        if col_list[row:].count(0) == cop.rows -1 -row:
            pivots.append((row,col))
            col +=1
            row +=1
            continue
        else:
            
            rep_ind = find_nonzero_ind(col_list,row) 
            multiple_factor = add_mult_row(cop,cop[row,col],cop[rep_ind,col])
            cop[rep_ind,:] += cop[row,:]*multiple_factor
    reduce_row(cop,pivots)
    return cop

def reduce_row(mtx,pivot_lst):
    pivot_lst.reverse()
    for row,col in pivot_lst:
        mtx[row,:] = mtx[row,:]/mtx[row,col]
        col_list = list(mtx[:,col])
        while col_list.count(0) < mtx.rows -1:
            rep_ind = find_nonzero_ind(col_list,row,True) 
            multiple_factor = add_mult_row(mtx,mtx[row,col],mtx[rep_ind,col])
            mtx[rep_ind,:] += mtx[row,:]*multiple_factor
            col_list = list(mtx[:,col])

    

    


If you are at all interesting in some serious testing of this script, the below will run this collection of functions 200 times on randomly generated 7x8 matrices and compare the resulting RREF to established RREF functions from the Sympy computer algebra package. 

In [3]:

def main ():
    serious_trbl_mkrs = []
    runs = 200
    for i in range(runs):
        trbl_mkr = Matrix(7,8,[random.randrange(1,100) if random.randrange(1,5) < 3 else 0 for num in range(56)])
        if shortEF(trbl_mkr) != trbl_mkr.rref()[0]:
            serious_trbl_mkrs.append(trbl_mkr)

    pprint(serious_trbl_mkrs)
    if len(serious_trbl_mkrs)/runs == 0.0:
        print('no incorrect RREFS')

if __name__ == "__main__":
    main()
    #ipdb.runcall(main)O
    

KeyboardInterrupt: 